# Setup Enviorment

In [7]:
from Preprocessing.selfsupervised_data_praparation import plot_samples, get_dataset_list
from Preprocessing.generate_embeddings import generate_embeddings_df, save_embeddings_as_csv
from tensorflow.keras import Sequential
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50V2

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd 

In [8]:
# Path to Dataset
path = '../Dataset_10_best_cities/'
model_name = 'ResNet50V2' #'VGG16' #'MobileNetV2' # 'vit' # 'autoencoder' #'variational_autoencoder'
latent_dim = 1024

target_size = (224, 224, 3)
model_input = (224, 224, 3)
band = None

# Model path
model_path = f'Weights/{model_name}_{target_size[0]}_{latent_dim}.h5'  

if target_size[2] == 1:
    # Embeddings path
    embeddings_path = f'Embeddings/{model_name}_Per_Band/{target_size[0]}_band{band+1}.csv'
else:
    # Embeddings path
    embeddings_path = f'Embeddings/{model_name}_{target_size[0]}_{latent_dim}.csv'

## Read dataset

In [9]:
#plot_samples(path)

In [10]:
image_list = get_dataset_list(path, show_dirs=True, head=0)

Images in directories: 
54001
41001
5001
50001
68001
8001
23001
76001
73001
5360


## Load Model

In [11]:
tf.random.set_seed(0)

if model_name == 'variational_autoencoder':
    from Models import Variational_Autoencoder
    model = Variational_Autoencoder.get_Variational_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim, lr=0.0001)
elif model_name == 'autoencoder':
    from Models import Autoencoder
    model = Autoencoder.get_Autoencoder(model_path=model_path, backbone=True, target_size = model_input, latent_dim = latent_dim)
elif model_name == 'vit':
    from Models import ViT
    model = ViT.get_vit_backbone(model_input)
elif model_name == 'MobileNetV2':
    cnn = MobileNetV2(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'VGG16': # min depth
    cnn = VGG16(input_shape=model_input, include_top=False, weights='imagenet')
elif model_name == 'ResNet50V2':
    cnn = ResNet50V2(input_shape=model_input, include_top=False, weights='imagenet') 

if model_name in ['MobileNetV2', 'VGG16', 'ResNet50V2']:
    model = Sequential()
    model.add(cnn)
    model.add(tf.keras.layers.GlobalAveragePooling2D())

    # freeze:
    for layer in model.layers:
        layer.trainable = False
        
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
Total params: 2,257,984
Trainable params: 0
Non-trainable params: 2,257,984
_________________________________________________________________


## Generate embeddings

In [12]:
if target_size[2] == 1:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size, BAND=band)
else:
    embeddings = generate_embeddings_df(image_list=image_list, model=model, target_size=target_size)
embeddings

,Municipality Code,Date,Embedding
0,54001,2017-07-30,"[0.884862, 0.008559655, 0.041577466, 0.1945201..."
1,54001,2015-12-20,"[1.5391555, 0.050785277, 0.0, 0.42412716, 0.08..."
2,54001,2017-11-19,"[0.9023423, 0.0, 0.105725676, 0.63589084, 0.0,..."
3,54001,2016-11-20,"[0.7258883, 0.028836785, 0.0, 0.9047181, 0.0, ..."
4,54001,2017-07-16,"[0.884862, 0.008559655, 0.041577466, 0.1945201..."
...,...,...,...
1635,5360,2017-04-16,"[1.0636309, 0.019519776, 0.3651379, 0.64856726..."
1636,5360,2018-12-02,"[0.18469608, 0.30143306, 0.0, 0.32649544, 0.0,..."
1637,5360,2018-05-20,"[1.9067419, 0.04090025, 0.0, 0.24894843, 0.000..."
1638,5360,2016-09-18,"[1.3758956, 0.0, 0.0, 0.15256377, 0.0, 0.25617..."


In [7]:
save_embeddings_as_csv(df=embeddings, path=embeddings_path)

In [8]:
pd.read_csv(embeddings_path)

,Municipality Code,Date,0,1,2,3,4,5,6,7,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,54001,2017-07-30,0.884862,0.008560,0.041577,0.194520,0.000000,1.437441,0.000000,0.276979,...,2.342425,0.055374,0.108854,0.000000,0.000000,0.014108,0.0,0.0,0.617788,0.230316
1,54001,2015-12-20,1.539155,0.050785,0.000000,0.424127,0.081332,0.719962,0.000000,0.124466,...,2.358550,0.502655,0.000000,0.014720,0.000000,0.000000,0.0,0.0,0.072344,0.262443
2,54001,2017-11-19,0.902342,0.000000,0.105726,0.635891,0.000000,1.310283,0.010937,0.531591,...,2.683626,0.049845,0.040585,0.000000,0.000000,0.000965,0.0,0.0,0.893814,0.334269
3,54001,2016-11-20,0.725888,0.028837,0.000000,0.904718,0.000000,0.200825,0.010728,0.106947,...,1.905462,0.015076,0.000000,0.069759,0.000000,0.000000,0.0,0.0,0.000000,1.259045
4,54001,2017-07-16,0.884862,0.008560,0.041577,0.194520,0.000000,1.437441,0.000000,0.276979,...,2.342425,0.055374,0.108854,0.000000,0.000000,0.014108,0.0,0.0,0.617788,0.230316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,5360,2017-04-16,1.063631,0.019520,0.365138,0.648567,0.000000,0.973078,0.008396,0.070921,...,2.068055,0.009155,0.020599,0.000000,0.001912,0.000000,0.0,0.0,0.926695,0.244764
1636,5360,2018-12-02,0.184696,0.301433,0.000000,0.326495,0.000000,0.685937,0.000000,0.000000,...,2.543913,0.022238,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.297692
1637,5360,2018-05-20,1.906742,0.040900,0.000000,0.248948,0.000955,0.025194,0.000000,0.970297,...,1.705161,0.113489,0.000000,0.012971,0.000000,0.000000,0.0,0.0,0.000000,0.937938
1638,5360,2016-09-18,1.375896,0.000000,0.000000,0.152564,0.000000,0.256177,0.000000,0.447673,...,2.413642,0.452365,0.000000,0.004762,0.000000,0.000000,0.0,0.0,0.000000,0.917825
